In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()

import sys
import os
import glob

ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
PASCAL_VOC_DATA = '/gpfs01/bethge/data/PascalVOC'
MASK_RCNN_MODEL_PATH = '/gpfs01/bethge/home/iustyuzh/siamese-mask-rcnn/Mask_RCNN/'
SIAMESE_MASK_RCNN_PATH = '/gpfs01/bethge/home/iustyuzh/siamese-mask-rcnn/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
import utils as siamese_utils
import model as siamese_model
import siamese_mrcnn_models as model_zoo
    
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle
from IPython import display
from collections import defaultdict

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class TrainConfig(coco.CocoConfig):
    # Reduced image sizes
    TARGET_MAX_DIM = 96
    TARGET_MIN_DIM = 75
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    
config = TrainConfig()
config.display()

In [ ]:
coco_val = siamese_utils.IndexedCocoDataset()
coco_val.load_coco(COCO_DATA, "val", year="2017")
coco_val.prepare()
coco_val.build_indices()
coco_val.ACTIVE_CLASSES = np.array(range(1,81))

In [ ]:
image_ids = coco_val.image_ids

In [ ]:
from choose_evaluation_instances import run_insatances_selection

In [ ]:
run_insatances_selection(coco_val, config, '/gpfs01/bethge/share/instances_evaluation/coco')

In [ ]:
import glob

In [ ]:
len(glob.glob('/gpfs01/bethge/share/instances_evaluation/coco/*'))

In [ ]:
plt.figure(figsize=(10, 5))

for i, image_id in enumerate(image_ids):
    coco_image_id = coco_val.image_info[image_id]["id"]
    
    save_filename = '{}.pkl'.format(coco_image_id)
    if os.path.isfile(save_filename):
        continue
    
    coco_val_instances = {}
    image = coco_val.load_image(image_id)
    
    gt_class_ids = coco_val.load_mask(image_id)[1]
    if not np.any(gt_class_ids > 0):
            continue
            
    categories = np.unique(gt_class_ids)
    _idx = categories > 0
    categories = categories[_idx]
    
    for category in categories:
        while True:
            try:
                target, target_size = siamese_utils.get_one_target(category, coco_val, config, return_original_size=True)
            except:
                print('Error extracting target. Imade ID {}, category {}'.format(image_id, category))
                
            if max(target_size[:2]) < 20:
                continue

            plt.subplot(1,2,1)
            plt.imshow(target)
            plt.title(coco_val.class_names[category], fontsize=18)
            plt.axis('off')

            plt.subplot(1,2,2)
            plt.imshow(image)
            plt.title('Input image', fontsize=18)
            plt.axis('off')

            display.display(plt.gcf())
            
            target_chosen = False

            while True:
                user_input = input()

                if user_input == 'n':
                    target_chosen = False
                    display.clear_output(wait=True)
                    break
                elif user_input == 'y':
                    print('Accepted. Imade ID {}, category {}'.format(coco_image_id, category))
                    coco_val_instances[category] = target
                    target_chosen = True
                    display.clear_output(wait=True)
                    break
                else:
                    print('Type y/n to accept/reject the target')
                    
            if target_chosen:
                break
                
    with open(save_filename, 'wb') as f:
        pickle.dump(coco_val_instances, f)

In [ ]:
target_size

In [ ]:
plt.imshow(coco_val_instances[532481][1])

In [ ]:
plt.imshow(coco_val.load_image(image_id))

In [ ]:
coco_val.class_names[category]

In [ ]:
plt.imshow(target)

In [ ]:
target.shape

In [ ]:
coco_image_ids = [coco_val.image_info[id]["id"] for id in image_ids]

In [ ]:
_, _, gt_class_ids, _, _ = modellib.load_image_gt(coco_val, config, 0, augmentation=False, use_mini_mask=config.USE_MINI_MASK)

In [ ]:
gt_class_ids

In [ ]:
coco_val.load_mask(0)

In [ ]:
image = coco_val.load_image(10)

In [ ]:
plt.imshow(image)